## Traitement de Données Massives

# Projet Partie n°2

### Binôme : Sapy Oscar & Berthillon Mickaël

# 0 : Création de l'arborescence

In [ ]:
import os

folders = ['metadata', 'images', 'prefered_image']

for folder in folders:
    if not os.path.exists(folder):
        os.makedirs(folder)
        print(f"Le dossier '{folder}' a été créé.")
    else:
        print(f"Le dossier '{folder}' existe déjà.")

# 1 : Lancement de la stack docker (download, extract, analyze)

In [ ]:
!docker-compose up --build

# 2 : Affiche l'image préférée sortie de docker_random

In [ ]:
import os
import tkinter as tk
from PIL import Image, ImageTk

class ImageDisplayer(tk.Tk):
    def __init__(self, preferred_folder):
        super().__init__()

        self.preferred_folder = preferred_folder
        self.images = os.listdir(preferred_folder)

        self.title("Preferred Image")
        self.geometry("500x500")

        if self.images:
            # Display only the first image
            image_file = self.images[0]
            image_path = os.path.join(self.preferred_folder, image_file)
            self.display_image(image_path)
        else:
            tk.Label(self, text="No preferred image found").pack()

    def display_image(self, image_path):
        image = Image.open(image_path)
        image.thumbnail((400, 400))
        photo = ImageTk.PhotoImage(image)

        self.image_label = tk.Label(self, image=photo)
        self.image_label.image = photo
        self.image_label.pack()

        tk.Label(self, text="Here is your preferred image").pack()

preferred_folder = "./preferred_image"
app = ImageDisplayer(preferred_folder)
app.mainloop()

# Commentaires
## Changements : chaque partie est désormais dockerisée, sauf l'affichage de tkinter car cela ne marche pas toujours d'utiliser le serveur X11 pour prendre contrôle du graphique à partir d'un conteneur Docker.

## Désormais, nous voulions que le processus ressemble à ceci :

### DOCKER_COMPOSE {
###     docker_download : télécharge le nb d'images voulues et les stocke dans le dossier
###     docker_extract  : extrait les metadonnées déjà présentes dans les images et les ajoute dans les fichiers respectifs .json dans /metadata
###     docker_analyse  : ajoute de nouveaux tags aux images en utilisant des librairies de reconnaissance
###     docker_random   : choisi une image aléatoire parmi celles disponibles et la déplace dans le dossier preferred_image
### }
### Enfin, il est possible d'afficher l'image préférée à l'aide de tkinter et du script python ci-dessus.

## Parallélisation : images_rdd.map(extract_metadata).collect()
## Nous parallélisons au sein du docker_extract l'extraction des metadonnées sur les différents fichiers en même temps grâce à map.